# Libraries

In [1]:
import pandas as pd
import pyodbc
import os
from datetime import datetime
from openpyxl import load_workbook
import shutil

# Import Data

In [2]:
%%time
weeks = '03_06'
path_so_st = '../../../../../../../genommalabinternacional/Cristian Javier Sanchez Yepez - NoB2B/Copservir/Copservir Faltante/consolidado_2021.xlsx'
path_catal = '../../../../../../02Paises/Colombia/Copservir/'

df = pd.read_excel(path_so_st, encoding='latin1')
df_pos = pd.read_excel(path_catal + 'Maestra Locales Copservir.xlsx', encoding='latin1')
df_prices = pd.read_excel(path_catal + 'Maestra Precios Copservir.xlsx',encoding='latin1')

Wall time: 13.9 s


# Clean Data & Merge

In [3]:
df.rename({'Código producto':'Cod_Hom'}, axis=1, inplace=True)

df_pos.rename({'PDV':'cod'}, axis=1, inplace=True)

In [4]:
# Cruzamos ambas maestras con los datos de SO y Stock
final = pd.merge(pd.merge(df, df_pos, on='cod', how='left'), df_prices, on='Cod_Hom', how='left')

In [5]:
# Agregamos el Monto de las Ventas
final['Costo_SO'] = final['so']*final['Precio']

# Agregamos el Monto del Stock
final['Costo_Stock'] = final['Inv']*final['Precio']

In [6]:
# Creamos 2 dfs para guardar los datos necesarios

SO = final[['Fecha_Venta', 'Cadena', 'Cod_Prod', 'EAN', 'Descripcion', 'cod', 'NOMBRE PDV', 'so', 'Costo_SO']].copy()

Stock = final[['Fecha_Venta', 'Cadena', 'Cod_Prod', 'EAN', 'Descripcion', 'cod', 'NOMBRE PDV', 'Inv', 'Costo_Stock']].copy()

# Export data

In [7]:
def write_layout(filename, df, sellout=True):
    writer = pd.ExcelWriter(filename, engine='openpyxl')
    writer.book = load_workbook(filename)
    writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    if sellout == True:
        df.to_excel(writer, sheet_name='Ventas', startcol=1, startrow=18, index=False, header=None)
    else:
        df.to_excel(writer, sheet_name='Stock', startcol=1, startrow=18, index=False, header=None)
    writer.save()

In [8]:
%%time
# Creamos un nuevo folder en donde agregamos el layout para exportar a ISV
original = '../../../1Data/2Catalogue/toISV/Formato No B2B Orig.xlsx'
new_folder = path_catal + '/S' + weeks
try:
    os.mkdir(new_folder)
except:
    pass

# Exportamos los datos al layout de ISV
filename = 'Formato No B2B COPSERVIR 2021_' + weeks
path_export = new_folder + '/' + filename +'.xlsx'
shutil.copy(original, path_export)
write_layout(path_export, SO, True)
write_layout(path_export, Stock, False)

Wall time: 5min 50s
